# Problem Description.

The task of this project is to solve the shift design problem in ASP.
The goal is to find shifts 
that meet the required number of employees at every time slot of a given schedule.

The input
specifies the number of days of the schedule, 
the number of time slots per day,
and the number of required employees for each slot of each day.
It also includes a description of shift types.
Each type has a minimum starting slot, a maximum starting slot, 
a minimum length and a maximum length.
Additionally, the input specifies and exceed and a shorten number, 
that indicate how many employees 
can be above or below the required number at each time slot, respectively.
In practice, the problem asks for the minimization of objectives like the number of shifts, 
the sum of the deviations from the required employees per slot, or some combination of them. 
In our case, we do not focus on such optimizations but impose an upper limit on the number of shifts.

A solution is a set of shifts whose size does not exceed the given limit.
Each shift has a starting slot, a length, 
and a number of employees assigned for each day.
The starting slot and the length must align with one (or more) of the shift types, 
i.e.,
the starting slot must fall between the minimum and the maximum starting slot of the type, 
and the length must fall between its minimum and maximum lengths.
Moreover, the length cannot be greater than the result of multiplying the number of slots by the number of days.
The number of assigned employees may vary across days, 
but must range between 1
and the maximum number of employees required at any single slot. 

Each shift starts daily at its designated starting point, 
and spans over a consecutive number of slots equal to its length.
If a shift extends beyond the end of a day, it continues into the next day.
Note that the schedule is cyclic: 
the next day of the last day of the schedule is the first day of the schedule.
As an example, if there are 3 days and 8 slots:
* A shift of length 3 starting at slot 8 of day 2 spans over that slot and slots 1 and 2 of day 3.
* A shift of length 24 starting at slot 8 of day 2 spans over that slot through all slots of days 3 and
  day 1, until slot 7 of day 2; in other words, it spans over all slots.

The number of employees working at a given day and time slot is 
the sum of the employees assigned to shifts spanning over that specific slot.
This total number must not exceed or fall below the expected number of employees 
by more than the exceed and the shorten numbers of the input, respectively. 
As an example, if at time slot 8 of day 2 a number of two employees are required:
* If both exceed and shorten are 0, then exactly 2 employees must be working at that slot.
* If exceed is 1 and shorten is 0, then either 2 or 3 employees must be working at that slot.
* If exceed is 0 and shorten is 1, then either 1 or 2 employees must be working at that slot.


The following image represents one problem instance along with its unique solution:
* There is a single day with 8 time slots. In the image, every slot corresponds to one column. 
* The asterisks in each column represent the number of employees required at that slot.
  For example, at slots 1 and 2 only one employee is required, 
  while at slots 5 and 6 the instance requires 5 employees.

<img width="400" height="250" src="img/shifts.png"/>

The instance specifies some additional information not shown in the image: 
* There can be no deviation from the required number of employees at any time slot,
  i.e., the exceed and shorten numbers are both zero.
* There are three shift types: 
  they must start exactly at slot 3, 5 and 8, respectively,
  and they all have length 1 to 4. 
* The maximum number of shifts is 3.

The solution is represented by the colors:
* One shift (in red) starts at slot 3, has a length of 4, and 3 employees assigned.
* Another shift (in orange) starts at slot 5, has a length of 4, and 2 employees assigned.
* The other shift (in yellow) starts at slot 8, has a length of 4, and 1 employee assigned.

Observe how the shift in yellow starts at the last slot and continues from the first one.

See also how the solution provides the required number of employees at every time slot.
For example:
* at slots 1 and 2 the yellow shift provides the unique required employee, 
and
* at slots 5 and 6 the other two shifts provide the 5 required employees.

The solution is also valid for any value of the exceed and the shorten numbers. 
* On the other hand, if slot 3 required only 3 employees, then the solution would no longer be valid, since slot 3 would be overstaffed. However, the solution would be valid if the exceed number was 1.
* If the yellow shift had only length 3, then the solution shown above would no longer be valid since slot 3 would be understaffed. However, the solution would be valid if the shorten number was 1.

This project is based on the article:
* [Abseher, M., Gebser, M., Musliu, N., Schaub, T., & Woltran, S. (2016). Shift Design with Answer Set Programming. Fundam. Inform., 147(1), 1–25.](https://www.cs.uni-potsdam.de/wv/publications/DBLP_journals/fuin/AbseherGMSW16.pdf)

# Representation in ASP.

An instance of the problem is represented by facts of the following predicates:
```
  day(D).            % D is a day of the schedule
  slot(S).           % S is a time slot
  emp(D,S,E).        % at day D and slot S a number of E employees are required
  exceed(EX).        % the required number of employees can be  exceeded by EX
  shorten(SH).       % the required number of employees can be shortened by SH
  type(S1,S2,L1,L2). % there is a shift type with minimum starting time S1, maximum
                     % starting time S2, minimum length L1 and maximum length L2
  maxshifts(M).      % there can be at most M shifts
```

All arguments are integers greater or equal to `1`, except `EX` and `SH` that are integers greater or equal to `0`. 


The example shown before is represented by the following facts:
```
day(1).
slot(1..8).
emp(1,1,1).
emp(1,2,1).
emp(1,3,4).
emp(1,4,3).
emp(1,5,5).
emp(1,6,5).
emp(1,7,2).
emp(1,8,3).
exceed(0).
shorten(0).
type(3,3,1,4).
type(5,5,1,4).
type(8,8,1,4).
maxshifts(3).
```

A solution is a set of shifts represented by atoms of the predicate `shift/4`:
```
  shift(S,L,D,E).  % there is a shift that starts at slot S with length L,
                   % and at day D it has E employees assigned
```
The arguments `S` and `L` define a shift:
if `shift(S,L,D,E)` belongs to an answer set then 
for every day `DD` there must be exactly one atom 
of the form `shift(S,L,DD,EE)` (where `EE >= 1`) in the answer set.

For instance, the solution of the example consists of the following atoms:
```
shift(3,4,1,3) shift(5,4,1,2) shift(8,4,1,1)
```

# Task.

Write an ASP encoding such that given an instance,
the answer sets of the encoding and the instance
correspond to the solutions of the problem represented by the instance.

# Framework.

The directory ``asp`` contains the files that you need for the project. In the directory ``asp/instances`` you can find the instances (our example is `instance-03.lp`), and in the directory ``asp/solutions`` you can find their solutions in ``json`` format. 

You have to submit a file named ``shifts.lp``, included as a template in the directory ``asp``, that contains the following line (and no more ``#show`` statements) so that in the output only the atoms of predicate ``shift/4`` appear:

``#show shift/4.``

You can check if your encoding solves correctly all instances by running the ``Python`` script ``test.py`` as follows:
* ``python asp/test.py -e asp/shifts.lp -i asp/instances -s asp/solutions -t 100``

In this case, the timeout for each instance is set to `100` seconds, but you can use any other value instead.
We have tested the script with `Python` version `3.8`.

We recommend you to work locally in your computer, using your own installation of ``clingo``.

You can also run your encoding in the next cell. If you are using ``Binder``, remember to download to your computer the files that you modify, otherwise you will lose your changes.

In [ ]:
%%clingo 0 asp/instances/instance-03.lp -


%
% Guess solution candidates
%


%
% Check solution candidates
%


%
% Display
%

#show shifts/4.

# Visualization

You can use clingraph to visualize the solutions of the problem:
* https://github.com/potassco/clingraph

Clingraph can be installed using conda:
* `conda install -c potassco clingraph`

To visualize the solution of our example, you can run this command:
```
clingo asp/shifts.lp asp/instances/instance-03.lp asp/show.lp 0 --outf=2 | clingraph --view --dir='img' --format=png --out=render --prefix=viz_ --engine=neato --default-graph=shifts --viz-encoding=asp/viz.lp --name-format=model_{model_number}
```

The visualization program [asp/viz.lp](asp/viz.lp) generates images that are simpler than the one from above, 
but it can still be helpful to develop your encoding. 
You can also extend the visualization program to generate fancier images, like the one above.
The file [asp/viz.lp](asp/viz.lp) already contains some hints for doing this.

# Formalities.
You can work on the solution alone or in groups of two people. 
Different groups have to submit different solutions, in case
of plagiarism all groups involved will fail the project. 

Your solution has to correctly encode all solutions for every instance. 
This is tested automatically by the script ``test.py``. 

We will send you further instructions about the submission process from Moodle.

# Tips:

* If you are stuck you can contact us. We will do out best to answer all your questions. You can send us questions and remarks either via Moodle or by email.

* Start as soon as possible to avoid running out of time. However, if you still realize that you have problems making it before the deadline, please contact us instead of copying another solution.